# python super()
一想到 `super()` 就想到可以調用父類的方法嗎？

來認識真正的 `super()` 吧！ 

In [4]:
# 如果只是調用 Parent class 的方法，這樣寫有什麼不一樣！？
class Parent(object):
    def __init__(self):
        print('hello world')

class ChildA(Parent):
    def __init__(self):
        print('hello A')
        Parent.__init__(self)

class ChildB(Parent):
    def __init__(self):
        print('hello B')
        super().__init__()

ChildA()
ChildB()

hello A
hello world
hello B
hello world


In [6]:
# 假設遇到多重繼承的情況
class Dependency(Parent):
    def __init__(self):
        print("dependency")
        super().__init__()

class UserA(ChildA, Dependency):
    def __init__(self):
        print("User A")
        super().__init__()
        
class UserB(ChildB, Dependency):
    def __init__(self):
        print("User B")
        super().__init__()
        
UserA()  # ChildA 沒有 super() Parent
print('-'*10)
UserB()  # ChildB 沒有 super() Parent

User A
hello A
hello world
----------
User B
hello B
dependency
hello world


# why UserA didn't print 'dependency'
雖看在 ChildA, ChildB 時看起來很簡單，只是調用父類的方法，

但或許應該說，是剛好 `super()` 剛好調用 Parent class，

來看看 UserA 與 UserB 的打印流程：

* UserA: UserA() -> ChildA() -> Parent()
* UserB: UserB() -> ChildB() -> Dependency() -> Parent()

很明顯的差別在 ChildA() 並沒有使用 `super()` 造成這個差別！

其實 `super()` 跟調用父類並沒有實質上的關聯，

所以 `super()` 到底做了什麼呢? 而又為什麼 UserB 的打印流程不是先打印 hello world 在打印 dependency 呢？

In [9]:
# 詳細的看看調用流程

class Base(object):
    def __init__(self):
        print('Base!!')

class A(Base):
    def __init__(self):
        print('A in')
        super().__init__()
        print('A out')

class B(Base):
    def __init__(self):
        print('B in')
        super().__init__()
        print('B out')

class C(A, B):
    def __init__(self):
        print('C in')
        super().__init__()
        print('C out')

C()

C in
A in
B in
Base!!
B out
A out
C out


## C 繼承了 A B，而 A B 都繼承了 Base
打印流程:
```
C(in) -super()-> A(in) -super()-> B(in) -super()-> Base() -> B(out) -> A(out) -> C(out)
```
為什麼一路調用 super() 到了 A(in) 調用 super() 時，不是去調用 ***A()的父類Base()*** 而是轉而去調用 B() 呢？

# MRO 列表

原因就是在定義 class 時，python 會利用 ***方法解析顺序(Method Resolution Order, MRO)*** ，來計算繼承的 order，

簡單的說，就是會在你繼承來繼承去的過程，給定一個長幼排序，可以利用 `mro()` 來獲得這個排序。

也就是說一個 class 的 MRO 列表就是根據這個 class 的所有 Parents class 來排列出來的順序！

而這個排列有幾個大方向:

1. 子類永在父類前面
2. 如果下一層父類有多個合理選擇，就選先遇到的那個
2. 如果有多個父類，會根據 MRO 的順序做檢查

In [11]:
C.mro()

[__main__.C, __main__.A, __main__.B, __main__.Base, object]

# super() 原理

```python
def super(cls, inst):
    mro = inst.__class__.mro()
    return mro[mro.index(cls) + 1]
```
cls 代表 class; inst 代表 instance，然後做了兩件事:

1. 查詢這個 instance 的 MRO 列表
2. 然後找到此 class 在列表中的 index，並且返回下一個 class

於是我們回到上一個例子:

C(in) 後 super() 為 A()

A(in) 後 super() 為 B() ... 而不是所謂的父類 Base

然後以此類推就會得到結果！
